## `buenrostro`

In [1]:
import os
import gc
import h5py
import psutil
import pandas as pd
import scanpy as sc
from scipy import sparse

# Load data

In [2]:
# Set-up the paths
input_dir = '/cellar/users/aklie/data/ml4gland/Buenrostro_2018/processed/21Sep23/'
h5_file = os.path.join(input_dir, 'pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5')  # TODO
bed_file = os.path.join(input_dir, 'pbmc_granulocyte_sorted_3k_atac_peaks.bed')  # TODO
output_dir = '/cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/Buenrostro_2018/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# Read peaks and h5ad
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'], comment='#')
ad = sc.read_10x_h5(h5_file, gex_only=False)

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


# Filter low quality cells

In [4]:
ad_rna = ad[:, ad.var['feature_types'] == 'Gene Expression']
ad_atac = ad[:, ad.var['feature_types'] == 'Peaks']
ad_atac.var['chr'] = peak['chr'].values
ad_atac.var['start'] = peak['start'].values
ad_atac.var['end'] = peak['end'].values

# basic stats
sc.pp.filter_cells(ad_rna, min_genes=0)
sc.pp.filter_genes(ad_rna, min_cells=0)
sc.pp.filter_cells(ad_atac, min_genes=0)
sc.pp.filter_genes(ad_atac, min_cells=0)

# a gene need to be expressed in 5% cells
# a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad_rna = ad_rna[:, ad_rna.var['n_cells'] > thres]
ad_atac = ad_atac[:, ad_atac.var['n_cells'] > thres]

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/ipykernel_launcher.py:3: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  This is separate from the ipykernel package so we can avoid doing imports until
/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


# Save h5ad for next step

In [5]:
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad_atac = ad_atac[:, ad_atac.var['chr'].isin(chrs)]
ad_atac.write(os.path.join(output_dir, 'buen_atac_ad.h5ad'))

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


# Preprocess specifically for scBasset

In [2]:
%%bash
source activate scbasset
python /cellar/users/aklie/opt/ml4gland/scBasset/bin/scbasset_preprocess.py \
    --ad_file /cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/Buenrostro_2018/processed/buen_ad_sc.h5ad \
    --input_fasta /cellar/users/aklie/data/ml4gland/genomes/hg38/hg38.fa \
    --out_path /cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/Buenrostro_2018/processed

process 0 peaks takes 1.5 s
process 1000 peaks takes 2.1 s
process 2000 peaks takes 2.8 s
process 3000 peaks takes 3.5 s
process 4000 peaks takes 4.3 s
process 5000 peaks takes 5.0 s
process 6000 peaks takes 5.7 s
process 7000 peaks takes 6.4 s
process 8000 peaks takes 7.1 s
process 9000 peaks takes 7.9 s
process 10000 peaks takes 8.6 s
process 11000 peaks takes 9.4 s
process 12000 peaks takes 10.2 s
process 13000 peaks takes 10.9 s
process 14000 peaks takes 11.9 s
process 15000 peaks takes 12.7 s
process 16000 peaks takes 13.4 s
process 17000 peaks takes 14.0 s
process 18000 peaks takes 14.9 s
process 19000 peaks takes 15.6 s
process 20000 peaks takes 16.2 s
process 21000 peaks takes 17.1 s
process 22000 peaks takes 17.7 s
process 23000 peaks takes 18.5 s
process 24000 peaks takes 19.2 s
process 25000 peaks takes 20.1 s
process 26000 peaks takes 20.8 s
process 27000 peaks takes 21.7 s
process 28000 peaks takes 22.4 s
process 29000 peaks takes 23.2 s
process 30000 peaks takes 23.8 s
pr

Adding 1344 Ns to chr1:249080335-249080944
Adding 1344 Ns to chr1:249098835-249099737
Adding 1344 Ns to chr1:249101402-249101652
Adding 1344 Ns to chr1:249111000-249111707
Adding 1344 Ns to chr1:249116710-249116922
Adding 1344 Ns to chr1:249120026-249121090
Adding 1344 Ns to chr1:249123138-249123491
Adding 1344 Ns to chr1:249131994-249133450
Adding 1344 Ns to chr1:249152287-249152496
Adding 1344 Ns to chr1:249152806-249153938
Adding 1344 Ns to chr1:249157040-249157754
Adding 1344 Ns to chr1:249167069-249168712
Adding 1344 Ns to chr1:249183457-249184075
Adding 1344 Ns to chr1:249199927-249201313
Adding 1344 Ns to chr1:249218880-249219893
Adding 1344 Ns to chr1:249238986-249239393
Adding 1344 Ns to chr1:249240181-249240504
Adding 1344 Ns to chr10:133806367-133806778
Adding 1344 Ns to chr10:133836495-133837237
Adding 1344 Ns to chr10:133849545-133850610
Adding 1344 Ns to chr10:133878982-133880129
Adding 1344 Ns to chr10:133909774-133910202
Adding 1344 Ns to chr10:133955946-133956557
Addin

```bash

```

# Scratch